In [1]:
import os
import pickle
import statistics

In [2]:
R1MSUBS_TRAIN_PATH = os.path.abspath("./inputs/train_comments_subs.pkl")
R1MSUBS_VAL_PATH = os.path.abspath("./inputs/val_comments_subs.pkl")
R1MSUBS_TEST_PATH = os.path.abspath("./inputs/test_comments_subs.pkl")

IDX2WORD_PATH = os.path.abspath("./inputs/graph/idx_2_word.pkl")
WORD2IDX_PATH = os.path.abspath("./inputs/graph/word_2_idx.pkl")

with open(R1MSUBS_TRAIN_PATH, "rb") as train_file:
    r1msubs_train = pickle.load(train_file)
with open(R1MSUBS_VAL_PATH, "rb") as val_file:
    r1msubs_val = pickle.load(val_file)
with open(R1MSUBS_TEST_PATH, "rb") as test_file:
    r1msubs_test = pickle.load(test_file)

with open(IDX2WORD_PATH, "rb") as idx2word_file:
    idx2word = pickle.load(idx2word_file)
with open(WORD2IDX_PATH, "rb") as word2idx_file:
    word2idx = pickle.load(word2idx_file)


In [3]:
INGR_NODES_PATH = os.path.abspath("./inputs/graph/nodes_191120.csv")

rows = []
with open(INGR_NODES_PATH, 'r') as file:
    for line in file:
        values = line.split(",")

        rows.append(values)

node_idx_2_name = {}
for c, row in enumerate(rows):
    if row[3] != "ingredient":
        continue
    node_idx_2_name[c] = row[1]

print(list(node_idx_2_name.items())[:5])
print(node_idx_2_name[2865])

[(1, '1%_fat_buttermilk'), (2, '1%_fat_cottage_cheese'), (3, '10%_cream'), (4, '100%_bran'), (5, '10_inch_flour_tortilla')]
ground_beef


In [4]:
VOCABS_PATH = os.path.abspath("./inputs/vocab_ingrs.pkl")

with open(VOCABS_PATH, "rb") as vocabs_file:
    vocabs = pickle.load(vocabs_file)

ModuleNotFoundError: No module named 'inv_cooking'

In [5]:
train_subs = []
for train_sample in r1msubs_train:
    sub = train_sample["subs"]
    if sub not in train_subs:
        train_subs.append(sub)

val_subs = []
for val_sample in r1msubs_val:
    sub = val_sample["subs"]
    if sub not in val_subs:
        val_subs.append(sub)

test_subs = []
for test_sample in r1msubs_test:
    sub = test_sample["subs"]
    if sub not in test_subs:
        test_subs.append(sub)

In [6]:
test_subs_not_in_train = []
for test_sub in test_subs:
    if test_sub not in train_subs and test_sub not in test_subs_not_in_train:
        test_subs_not_in_train.append(test_sub)

test_sources_not_in_train = []
train_sources = [pair[0] for pair in train_subs]
for test_sub in test_subs:
    if test_sub[0] not in train_sources and test_sub[0] not in test_sources_not_in_train:
        test_sources_not_in_train.append(test_sub)

test_targets_not_in_train = []
train_targets = [pair[1] for pair in train_subs]
for test_sub in test_subs:
    if test_sub[0] not in train_targets and test_sub[0] not in test_targets_not_in_train:
        test_targets_not_in_train.append(test_sub)

In [7]:
val_subs_not_in_train = []
for val_sub in val_subs:
    if val_sub not in train_subs and val_sub not in val_subs_not_in_train:
        val_subs_not_in_train.append(val_sub)

val_sources_not_in_train = []
train_sources = [pair[0] for pair in train_subs]
for val_sub in val_subs:
    if val_sub[0] not in train_sources and val_sub[0] not in val_sources_not_in_train:
        val_sources_not_in_train.append(val_sub)

val_targets_not_in_train = []
train_targets = [pair[1] for pair in train_subs]
for val_sub in val_subs:
    if val_sub[0] not in train_targets and val_sub[0] not in val_targets_not_in_train:
        val_targets_not_in_train.append(val_sub)

In [8]:
print("***samples***")
print(len(r1msubs_train))
print(len(r1msubs_val))
print(len(r1msubs_test))
print("***subs***")
print(len(train_subs))
print(len(val_subs))
print(len(test_subs))
print("***test***")
print(len(test_subs_not_in_train))
print(len(test_sources_not_in_train))
print(len(test_targets_not_in_train))
print("***val***")
print(len(val_subs_not_in_train))
print(len(val_sources_not_in_train))
print(len(val_targets_not_in_train))

***samples***
49044
10729
10747
***subs***
23421
6895
6831
***test***
3778
323
675
***val***
3828
286
630


In [10]:
# get the number of test and val samples with unseen sources

test_samples_with_unseen_sources = []
test_samples_with_unseen_subs = []
for test_sub in r1msubs_test:
    if test_sub["subs"][0] in [sub[0] for sub in test_sources_not_in_train]:
        test_samples_with_unseen_sources.append(test_sub)
    if test_sub["subs"] in test_subs_not_in_train:
        test_samples_with_unseen_subs.append(test_sub)

val_samples_with_unseen_sources = []
val_samples_with_unseen_subs = []
for val_sub in r1msubs_val:
    if val_sub["subs"][0] in [sub[0] for sub in val_sources_not_in_train]:
        val_samples_with_unseen_sources.append(val_sub)
    if val_sub["subs"] in val_subs_not_in_train:
        val_samples_with_unseen_subs.append(val_sub)

print(len(test_samples_with_unseen_sources))
print(len(val_samples_with_unseen_sources))
print(len(test_samples_with_unseen_subs))
print(len(val_samples_with_unseen_subs))

324
291
3881
3958


## What is the number and fraction of seen/unseen samples in the test and train splits for each of the data sets? (also recipes)

In [16]:
arcfive_train_0 = os.path.abspath("./outputs/new_comments/arcelik_five_fold/train_comments_subs_0.pkl")
arcfive_test_0 = os.path.abspath("./outputs/new_comments/arcelik_five_fold/test_comments_subs_0.pkl")
arcfive_train_1 = os.path.abspath("./outputs/new_comments/arcelik_five_fold/train_comments_subs_1.pkl")
arcfive_test_1 = os.path.abspath("./outputs/new_comments/arcelik_five_fold/test_comments_subs_1.pkl")
arcfive_train_2 = os.path.abspath("./outputs/new_comments/arcelik_five_fold/train_comments_subs_2.pkl")
arcfive_test_2 = os.path.abspath("./outputs/new_comments/arcelik_five_fold/test_comments_subs_2.pkl")
arcfive_train_3 = os.path.abspath("./outputs/new_comments/arcelik_five_fold/train_comments_subs_3.pkl")
arcfive_test_3 = os.path.abspath("./outputs/new_comments/arcelik_five_fold/test_comments_subs_3.pkl")
arcfive_train_4 = os.path.abspath("./outputs/new_comments/arcelik_five_fold/train_comments_subs_4.pkl")
arcfive_test_4 = os.path.abspath("./outputs/new_comments/arcelik_five_fold/test_comments_subs_4.pkl")

arcfive_splits = [
    (
        arcfive_train_0,
        arcfive_test_0
    ),
    (
        arcfive_train_1,
        arcfive_test_1
    ),
    (
        arcfive_train_2,
        arcfive_test_2
    ),
    (
        arcfive_train_3,
        arcfive_test_3
    ),
    (
        arcfive_train_4,
        arcfive_test_4
    ),
]


def get_unseen_pairs_and_recipes(train_path, val_or_test_path):
    with open(train_path, "rb") as train_file:
        train_comments = pickle.load(train_file)
    with open(val_or_test_path, "rb") as other_file:
        other_comments = pickle.load(other_file)

    n_train_samples = len(train_comments)
    n_other_samples = len(other_comments)

    seen_subs = set([comment["subs"] for comment in train_comments])
    seen_recipe_ids = set([comment["id"] for comment in train_comments])

    distinct_other_subs = set([comment["subs"] for comment in other_comments])
    distinct_other_ids = set([comment["id"] for comment in other_comments])


    unseen_subs = []
    unseen_recipe_ids = []
    for comment in other_comments:
        sub = comment["subs"]
        id = comment["id"]

        if sub not in seen_subs and sub not in unseen_subs:
            unseen_subs.append(sub)
        if id not in seen_recipe_ids and id not in unseen_recipe_ids:
            unseen_recipe_ids.append(id)


    n_samples_with_unseen_subs = 0
    n_samples_with_unseen_sources = 0
    seen_sources = [sub[0] for sub in seen_subs]

    for comment in other_comments:
        if comment["subs"] not in seen_subs:
            n_samples_with_unseen_subs += 1
        if comment["subs"][0] not in seen_sources:
            n_samples_with_unseen_sources += 1

    return unseen_subs, unseen_recipe_ids, distinct_other_subs, distinct_other_ids, n_train_samples, n_other_samples, n_samples_with_unseen_subs, n_samples_with_unseen_sources


In [17]:
unseen_per_split = [get_unseen_pairs_and_recipes(k[0], k[1]) for k in arcfive_splits]

for i, unseen in enumerate(unseen_per_split):
    print(f"for fold {i}")
    print(f"nr unseen subs: {len(unseen[0])}")
    print(f"nr unseen recipe: {len(unseen[1])}")
    print(f"nr distinct other subs: {len(unseen[2])}")
    print(f"nr distinct other ids: {len(unseen[3])}")
    print(f"nr total train samples: {unseen[4]}")
    print(f"nr total other samples: {unseen[5]}")
    print(f"nr of samples with unseen subs: {unseen[6]}")
    print(f"nr of samples with unseen sources: {unseen[7]}")
    print("*****")

for fold 0
nr unseen subs: 21
nr unseen recipe: 60
nr distinct other subs: 75
nr distinct other ids: 123
nr total train samples: 524
nr total other samples: 132
nr of samples with unseen subs: 22
nr of samples with unseen sources: 14
*****
for fold 1
nr unseen subs: 19
nr unseen recipe: 57
nr distinct other subs: 77
nr distinct other ids: 116
nr total train samples: 525
nr total other samples: 131
nr of samples with unseen subs: 20
nr of samples with unseen sources: 6
*****
for fold 2
nr unseen subs: 16
nr unseen recipe: 69
nr distinct other subs: 77
nr distinct other ids: 122
nr total train samples: 525
nr total other samples: 131
nr of samples with unseen subs: 18
nr of samples with unseen sources: 7
*****
for fold 3
nr unseen subs: 17
nr unseen recipe: 59
nr distinct other subs: 75
nr distinct other ids: 121
nr total train samples: 525
nr total other samples: 131
nr of samples with unseen subs: 17
nr of samples with unseen sources: 6
*****
for fold 4
nr unseen subs: 25
nr unseen rec

In [13]:
r1msubs_train = os.path.abspath("./inputs/train_comments_subs.pkl")
r1msubs_val = os.path.abspath("./inputs/val_comments_subs.pkl")
r1msubs_test = os.path.abspath("./inputs/test_comments_subs.pkl")

r1m_val_nums = get_unseen_pairs_and_recipes(r1msubs_train, r1msubs_val)
r1m_test_nums = get_unseen_pairs_and_recipes(r1msubs_train, r1msubs_test)

for i, unseen in enumerate([r1m_val_nums, r1m_test_nums]):
    label = "val" if i == 0 else "test"

    print(f"for fold {label}")
    print(f"nr unseen subs: {len(unseen[0])}")
    print(f"nr unseen recipe: {len(unseen[1])}")
    print(f"nr distinct other subs: {len(unseen[2])}")
    print(f"nr distinct other ids: {len(unseen[3])}")
    print(f"nr total train samples: {unseen[4]}")
    print(f"nr total other samples: {unseen[5]}")
    print("*****")

for fold val
nr unseen subs: 3828
nr unseen recipe: 6670
nr distinct other subs: 6895
nr distinct other ids: 6670
nr total train samples: 49044
nr total other samples: 10729
*****
for fold test
nr unseen subs: 3778
nr unseen recipe: 6662
nr distinct other subs: 6831
nr distinct other ids: 6662
nr total train samples: 49044
nr total other samples: 10747
*****


## What is the diversity and mean diversity for each of the 5 fold results set?

In [9]:
def calc_diversity(rows):
    all_ranks_for_sources = {}
    for row in rows:
        source = row[0]
        top_rank_pred = row[3]

        if source not in list(all_ranks_for_sources.keys()):
            all_ranks_for_sources[source] = []
        all_ranks_for_sources[source].append(top_rank_pred)

    undiverse_top_rank_count = 0
    undiverse_top_rank_count_with_more_than_10_recipes = 0
    abs_div_sum = 0
    distinct_source_count = 0

    for source, all_targets in all_ranks_for_sources.items():
        distinct_targets = []
        for target in all_targets:
            if target not in distinct_targets:
                distinct_targets.append(target)

        n_unique_preds = len(distinct_targets)
        distinct_source_count += 1
        abs_div_sum += n_unique_preds
        if n_unique_preds <= 1:
            undiverse_top_rank_count += 1
            if len(all_targets) > 10:
                undiverse_top_rank_count_with_more_than_10_recipes += 1

    avg_div = abs_div_sum / distinct_source_count

    return avg_div, undiverse_top_rank_count, undiverse_top_rank_count_with_more_than_10_recipes


    # print(f"average top rank diversity: {abs_div_sum / distinct_source_count}")
    # print(f"total number of distinct substitution sources: {abs_div_sum}")
    # print(f"number of substitution sources which have only 1 recommended ingredient: {undiverse_top_rank_count}")
    # print(f"number of substitution sources which have only 1 recommended ingreident and which is applied to more than 10 recipes: {undiverse_top_rank_count_with_more_than_10_recipes}")

def load_rows(file_path):
    # val_ranks_path = os.path.abspath("./val_ranks.txt")

    rows = []
    with open(file_path, 'r') as file:
        for line in file:
            values = line.split()

            rows.append(values)
    return rows

In [16]:
arcfive_base_path = os.path.abspath("./arc_five")

ds_baseline = []
ds_contrastive = []
ds_ring = []
ds_ingredients = []
ds_actions = []
ds_flow = []

dgt_baseline = []
dgt_contrastive = []
dgt_ring = []
dgt_ingredients = []
dgt_actions = []
dgt_flow = []

for i in range(5):
    baseline_suf = f"{i}/baseline_{i}_500/test_ranks.txt"
    contrastive_suf = f"{i}/negative_{i}_500/test_ranks.txt"
    ring_suf = f"{i}/ring_{i}_500/test_ranks.txt"
    ingredients_suf = f"{i}/ingredients_{i}_500/test_ranks.txt"
    actions_suf = f"{i}/actions_{i}_500/test_ranks.txt"
    flow_suf = f"{i}/flow_{i}_500/test_ranks.txt"

    baseline_path = os.path.join(arcfive_base_path, baseline_suf)
    contrastive_path = os.path.join(arcfive_base_path, contrastive_suf)
    ring_path = os.path.join(arcfive_base_path, ring_suf)
    ingredients_path = os.path.join(arcfive_base_path, ingredients_suf)
    actions_path = os.path.join(arcfive_base_path, actions_suf)
    flow_path = os.path.join(arcfive_base_path, flow_suf)

    baseline_rows = load_rows(baseline_path)
    contrastive_rows = load_rows(contrastive_path)
    ring_rows = load_rows(ring_path)
    ingredients_rows = load_rows(ingredients_path)
    actions_rows = load_rows(actions_path)
    flow_rows = load_rows(flow_path)

    baseline_div, baseline_d1, baseline_dgt = calc_diversity(baseline_rows)
    contrastive_div, contrastive_d1, contrastive_dgt = calc_diversity(contrastive_rows)
    ring_div, ring_d1, ring_dgt = calc_diversity(ring_rows)
    ingredients_div, ingredients_d1, ingredients_dgt = calc_diversity(ingredients_rows)
    actions_div, actions_d1, actions_dgt = calc_diversity(actions_rows)
    flow_div, flow_d1, flow_dgt = calc_diversity(flow_rows)

    ds_baseline.append(baseline_div)
    ds_contrastive.append(contrastive_div)
    ds_ring.append(ring_div)
    ds_ingredients.append(ingredients_div)
    ds_actions.append(actions_div)
    ds_flow.append(flow_div)

    dgt_baseline.append(baseline_dgt)
    dgt_contrastive.append(contrastive_dgt)
    dgt_ring.append(ring_dgt)
    dgt_ingredients.append(ingredients_dgt)
    dgt_actions.append(actions_dgt)
    dgt_flow.append(flow_dgt)

print("***")
print("D(1)")
print(f"  baseline: {statistics.fmean(ds_baseline)}$\pm${statistics.stdev(ds_baseline)}")
print(f"  contrastive: {statistics.fmean(ds_contrastive)}$\pm${statistics.stdev(ds_contrastive)}")
print(f"  ring: {statistics.fmean(ds_ring)}$\pm${statistics.stdev(ds_ring)}")
print(f"  ingredients: {statistics.fmean(ds_ingredients)}$\pm${statistics.stdev(ds_ingredients)}")
print(f"  actions: {statistics.fmean(ds_actions)}$\pm${statistics.stdev(ds_actions)}")
print(f"  flow: {statistics.fmean(ds_flow)}$\pm${statistics.stdev(ds_flow)}")

print("***")
print("Recipes with 1 sub pair and more than 10 recipes")
print(f"  baseline: {statistics.fmean(dgt_baseline)}$\pm${statistics.stdev(dgt_baseline)}")
print(f"  contrastive: {statistics.fmean(dgt_contrastive)}$\pm${statistics.stdev(dgt_contrastive)}")
print(f"  ring: {statistics.fmean(dgt_ring)}$\pm${statistics.stdev(dgt_ring)}")
print(f"  ingredients: {statistics.fmean(dgt_ingredients)}$\pm${statistics.stdev(dgt_ingredients)}")
print(f"  actions: {statistics.fmean(dgt_actions)}$\pm${statistics.stdev(dgt_actions)}")
print(f"  flow: {statistics.fmean(dgt_flow)}$\pm${statistics.stdev(dgt_flow)}")


***
D(1)
  baseline: 1.0510317705801777$\pm$0.03371856702877994
  contrastive: 1.0392366777534041$\pm$0.02416547129557241
  ring: 1.0553575079630535$\pm$0.03374610129495772
  ingredients: 1.0527557445202285$\pm$0.049658565895570383
  actions: 1.071915978457628$\pm$0.036535711643531435
  flow: 1.0667885433884376$\pm$0.05315412528271684
***
Recipes with 1 sub pair and more than 10 recipes
  baseline: 0.2$\pm$0.4472135954999579
  contrastive: 0.0$\pm$0.0
  ring: 0.0$\pm$0.0
  ingredients: 0.2$\pm$0.4472135954999579
  actions: 0.2$\pm$0.4472135954999579
  flow: 0.2$\pm$0.4472135954999579


## Diversity for ArcSubs with only unseen samples

In [14]:
# for set with unly unseen samples:
arcfive_base_path = os.path.abspath("C:/Users/David/project/gismo/out/arc_unseen")

ds_baseline = []
ds_contrastive = []
ds_ring = []
ds_ingredients = []
ds_actions = []
ds_flow = []

dgt_baseline = []
dgt_contrastive = []
dgt_ring = []
dgt_ingredients = []
dgt_actions = []
dgt_flow = []

for i in range(1, 4):
    baseline_suf = f"baseline_{i}_500/test_ranks.txt"
    contrastive_suf = f"negative_sampling_{i}_500/test_ranks.txt"
    ring_suf = f"ring_{i}_500/test_ranks.txt"
    ingredients_suf = f"ingredients_{i}_500/test_ranks.txt"
    actions_suf = f"simple_instructions_{i}_500/test_ranks.txt"
    flow_suf = f"complex_instructions_{i}_500/test_ranks.txt"

    baseline_path = os.path.join(arcfive_base_path, baseline_suf)
    contrastive_path = os.path.join(arcfive_base_path, contrastive_suf)
    ring_path = os.path.join(arcfive_base_path, ring_suf)
    ingredients_path = os.path.join(arcfive_base_path, ingredients_suf)
    actions_path = os.path.join(arcfive_base_path, actions_suf)
    flow_path = os.path.join(arcfive_base_path, flow_suf)

    baseline_rows = load_rows(baseline_path)
    contrastive_rows = load_rows(contrastive_path)
    ring_rows = load_rows(ring_path)
    ingredients_rows = load_rows(ingredients_path)
    actions_rows = load_rows(actions_path)
    flow_rows = load_rows(flow_path)

    baseline_div, baseline_d1, baseline_dgt = calc_diversity(baseline_rows)
    contrastive_div, contrastive_d1, contrastive_dgt = calc_diversity(contrastive_rows)
    ring_div, ring_d1, ring_dgt = calc_diversity(ring_rows)
    ingredients_div, ingredients_d1, ingredients_dgt = calc_diversity(ingredients_rows)
    actions_div, actions_d1, actions_dgt = calc_diversity(actions_rows)
    flow_div, flow_d1, flow_dgt = calc_diversity(flow_rows)

    ds_baseline.append(baseline_div)
    ds_contrastive.append(contrastive_div)
    ds_ring.append(ring_div)
    ds_ingredients.append(ingredients_div)
    ds_actions.append(actions_div)
    ds_flow.append(flow_div)

    dgt_baseline.append(baseline_dgt)
    dgt_contrastive.append(contrastive_dgt)
    dgt_ring.append(ring_dgt)
    dgt_ingredients.append(ingredients_dgt)
    dgt_actions.append(actions_dgt)
    dgt_flow.append(flow_dgt)

print("***")
print("D(1)")
print(f"  baseline: {statistics.fmean(ds_baseline)}$\pm${statistics.stdev(ds_baseline)}")
print(f"  contrastive: {statistics.fmean(ds_contrastive)}$\pm${statistics.stdev(ds_contrastive)}")
print(f"  ring: {statistics.fmean(ds_ring)}$\pm${statistics.stdev(ds_ring)}")
print(f"  ingredients: {statistics.fmean(ds_ingredients)}$\pm${statistics.stdev(ds_ingredients)}")
print(f"  actions: {statistics.fmean(ds_actions)}$\pm${statistics.stdev(ds_actions)}")
print(f"  flow: {statistics.fmean(ds_flow)}$\pm${statistics.stdev(ds_flow)}")

print("***")
print("Recipes with 1 sub pair and more than 10 recipes")
print(f"  baseline: {statistics.fmean(dgt_baseline)}$\pm${statistics.stdev(dgt_baseline)}")
print(f"  contrastive: {statistics.fmean(dgt_contrastive)}$\pm${statistics.stdev(dgt_contrastive)}")
print(f"  ring: {statistics.fmean(dgt_ring)}$\pm${statistics.stdev(dgt_ring)}")
print(f"  ingredients: {statistics.fmean(dgt_ingredients)}$\pm${statistics.stdev(dgt_ingredients)}")
print(f"  actions: {statistics.fmean(dgt_actions)}$\pm${statistics.stdev(dgt_actions)}")
print(f"  flow: {statistics.fmean(dgt_flow)}$\pm${statistics.stdev(dgt_flow)}")

***
D(1)
  baseline: 1.0337552742616032$\pm$0.03185584149903265
  contrastive: 1.0126582278481013$\pm$0.012658227848101222
  ring: 1.0379746835443038$\pm$0.012658227848101333
  ingredients: 1.0379746835443038$\pm$0.021924693766694637
  actions: 1.0421940928270041$\pm$0.02635020252488768
  flow: 1.0168776371308017$\pm$0.014616462511129715
***
Recipes with 1 sub pair and more than 10 recipes
  baseline: 0.0$\pm$0.0
  contrastive: 0.0$\pm$0.0
  ring: 0.0$\pm$0.0
  ingredients: 0.0$\pm$0.0
  actions: 0.0$\pm$0.0
  flow: 0.0$\pm$0.0


## Can we find some interesting recipes that show that diversity would be good?

In [52]:
# get all the subs pairs that had only one sub target with their recipes

gismo_baseline_val_rows_path = os.path.abspath("./outputs/gismo_baseline_results/val_ranks.txt")

gismo_baseline_rows = load_rows(gismo_baseline_val_rows_path)

recs_per_source = {}
for row_idx, row in enumerate(gismo_baseline_rows):
    source = row[0]
    gt_sub = row[1]
    rec_rank = row[2]
    top_rec_sub = row[3] # if you swap this out with the full ranks, that file only has the source and then the top ranked rec and all ranked recs starting from the top ranked rec (again)
    if source not in list(recs_per_source.keys()):
        recs_per_source[source] = []
    recs_per_source[source].append(top_rec_sub)

distinct_top_subs_for_common_sources = {}
for source, targets in recs_per_source.items():
    if len(targets) < 10:
        continue
    if source not in list(distinct_top_subs_for_common_sources.keys()):
        distinct_top_subs_for_common_sources[source] = []
    for target in set(targets):
        distinct_top_subs_for_common_sources[source].append(target)

undiverse_sources_with_target = {a:b for a, b in list(distinct_top_subs_for_common_sources.items()) if len(b) == 1}
print(len(undiverse_sources_with_target))

81


In [91]:
bad_sub_pair_names = []
for source, distinct_targets in undiverse_sources_with_target.items():
    target_names = [node_idx_2_name[int(distinct_target)] for distinct_target in distinct_targets if int(distinct_target) in list(node_idx_2_name.keys())]
    source_name = node_idx_2_name[int(source)]
    bad_sub_pair_names.append([source_name, target_names])

for names in bad_sub_pair_names:
    print(names)

['ground_clove', ['allspice']]
['margarine', ['butter']]
['maple_syrup', ['honey_syrup']]
['lime_juice', ['lemon_juice']]
['cream', ['milk']]
['olive_oil', ['butter']]
['sour_cream', ['yogurt']]
['vanilla_extract', ['almond_extract']]
['parsley', ['cilantro']]
['sweet_potato', ['yam']]
['chip', ['green_onion']]
['salt', ['garlic_salt']]
['lemon', ['lime']]
['plain_yogurt', ['sour_cream']]
['half_and_half', ['milk']]
['walnut', ['pecan']]
['lean_ground_beef', ['ground_turkey']]
['cottage_cheese', ['ricotta_cheese']]
['extra_virgin_olive_oil', ['butter']]
['vegetable_broth', ['chicken_broth']]
['applesauce', ['oil']]
['pecan', ['walnut']]
['ground_beef', ['ground_turkey']]
['stewed_tomato', ['rotel_tomato']]
['egg', ['egg_white']]
['cinnamon', ['pumpkin_pie_spice']]
['ground_pork', ['ground_turkey']]
['lettuce', ['spinach']]
['garlic_clove', ['garlic_powder']]
['corn_syrup', ['honey_syrup']]
['flour_tortilla', ['corn_tortilla']]
['apple', ['pear']]
['garlic_salt', ['garlic_powder']]
['le

In [55]:
# let's compare with the actions graph recommendations for the recipe1msubs dataset

action_graph_val_rows_path = os.path.abspath("./a_mode_results_inspections/simpleInstructionContext/val_ranks.txt")

action_graph_rows = load_rows(action_graph_val_rows_path)

action_recs_per_former_undiverse_source = {}
for row_idx, row in enumerate(action_graph_rows):
    source = row[0]
    gt_sub = row[1]
    rec_rank = row[2]
    top_rec_sub = row[3] # if you swap this out with the full ranks, that file only has the source and then the top ranked rec and all ranked recs starting from the top ranked rec (again)
    if source in list(undiverse_sources_with_target.keys()):
        if source not in list(action_recs_per_former_undiverse_source.keys()):
            action_recs_per_former_undiverse_source[source] = []
        if top_rec_sub not in action_recs_per_former_undiverse_source[source]:
            action_recs_per_former_undiverse_source[source].append(top_rec_sub)

more_diverse_action_sources = {a:b for a, b in list(action_recs_per_former_undiverse_source.items()) if len(b) > 1}
print(len(more_diverse_action_sources))

30


In [90]:
#which ingredients did better?
better_sub_pair_names = []
for source, distinct_targets in more_diverse_action_sources.items():
    target_names = [node_idx_2_name[int(distinct_target)] for distinct_target in distinct_targets if int(distinct_target) in list(node_idx_2_name.keys())]
    source_name = node_idx_2_name[int(source)]
    better_sub_pair_names.append([source_name, target_names])

for names in better_sub_pair_names:
    print(names)

['ground_clove', ['allspice', 'nutmeg']]
['cream', ['milk', 'buttermilk']]
['olive_oil', ['butter', 'applesauce', 'canola_oil', 'sesame_oil']]
['sour_cream', ['yogurt', 'vanilla_yogurt', 'milk']]
['salt', ['garlic_salt', 'sugar', 'sea_salt', 'soy_sauce']]
['vegetable_broth', ['chicken_broth', 'water']]
['stewed_tomato', ['rotel_tomato', 'diced_tomato']]
['egg', ['egg_white', 'applesauce']]
['cinnamon', ['pumpkin_pie_spice', 'allspice', 'apple_pie_spice', 'ground_ginger', 'chili_powder']]
['apple', ['pear', 'applesauce']]
['lemon_juice', ['lime_juice', 'orange_juice']]
['fish_sauce', ['soy_sauce', 'oyster_sauce']]
['green_chilies', ['jalapeno', 'fresh_jalapeno_chilies', 'salsa', 'rotel']]
['apple_cider', ['apple_juice', 'chicken_broth']]
['chicken_bouillon_cube', ['chicken_broth', 'chicken_stock_cube']]
['seasoning_salt', ['garlic_powder', 'garlic_salt', 'onion_salt', 'cajun_seasoning', 'creole_seasoning']]
['low_fat_cottage_cheese', ['ricotta_cheese', 'sour_cream']]
['low_fat_sour_crea

olive oil -> sesame oil conventionally might fit better if trying to cook in a east asian style, where the nutty sesame aroma adds a robustness to the taste, especially if butter is not a familiar ingredient
however, in an aglio et olio recipe butter might fit better because the less intense taste give the garlic more room to breathe

In [77]:
print("check out those interesting substitutions for olive oil, tho!!!")

2853
